In [24]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential

from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,BatchNormalization,RandomZoom,RandomRotation,RandomBrightness,Flatten,Rescaling,RandomContrast,Input, RandomTranslation


In [25]:
train=keras.utils.image_dataset_from_directory(
    directory="/kaggle/input/gender-classification-dataset/Training",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256,256)

)
valid=keras.utils.image_dataset_from_directory(
    directory="/kaggle/input/gender-classification-dataset/Validation",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256,256)

)

Found 47009 files belonging to 2 classes.
Found 11649 files belonging to 2 classes.


In [26]:

from keras.applications.vgg16 import VGG16,preprocess_input

vgg_model=VGG16(weights="imagenet",include_top=False,input_shape=(256,256,3))

In [27]:
def process(image,label):
    image=preprocess_input(image)
    return image,label
train=train.map(process)
valid=valid.map(process)

In [30]:
vgg_model.trainable = True

set_trainable = False

for layer in vgg_model.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

for layer in vgg_model.layers:
  print(layer.name,layer.trainable)

input_layer_5 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [31]:
argument=Sequential()

argument.add(RandomZoom(0.2))
argument.add(RandomRotation(0.2))
argument.add(RandomBrightness(0.2))
argument.add(RandomContrast(0.2))

In [33]:
model=Sequential()

model.add(Input(shape=(256,256,3)))

model.add(argument)
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dropout(.2))
model.add(Dense(64,activation="relu"))
model.add(Dropout(.2))
model.add(Dense(1,activation="sigmoid"))

In [35]:
model.compile(optimizer=keras.optimizers.Adam(1e-5), loss="binary_crossentropy", metrics=['accuracy'])

In [36]:
model.fit(train,epochs=15,validation_data=valid)

Epoch 1/15
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 375s 252ms/step - accuracy: 0.7323 - loss: 0.5801 - val_accuracy: 0.9084 - val_loss: 0.2300
Epoch 2/15
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 369s 251ms/step - accuracy: 0.8966 - loss: 0.2581 - val_accuracy: 0.8269 - val_loss: 0.3925
Epoch 3/15
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 369s 251ms/step - accuracy: 0.9177 - loss: 0.2109 - val_accuracy: 0.9188 - val_loss: 0.2043
Epoch 4/15
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 369s 251ms/step - accuracy: 0.9298 - loss: 0.1823 - val_accuracy: 0.9381 - val_loss: 0.1620
Epoch 5/15
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 369s 251ms/step - accuracy: 0.9359 - loss: 0.1713 - val_accuracy: 0.8817 - val_loss: 0.2840
Epoch 6/15
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 369s 251ms/step - accuracy: 0.9383 - loss: 0.1620 - val_accuracy: 0.8886 - val_loss: 0.2604
Epoch 7/15
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 368s 250ms/step - accuracy: 0.9420 - loss: 0.1537 - val_accuracy: 0.9125 - val_loss: 0.2123
Epoch 8/15
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 369s 251ms/step - ac

In [37]:
model.save("BEST_GENDER_CLASSIFICATION.keras")